# Hacking with nmap

Lets have some fun with nmap...

First some conventions, this notebook is using a Jupyter Notepook with an Python Kernel though most of the stuff we will be doing are bash commands. The Jupyter Puthon Kernel allows running of bash commands by prepending a exclemation point in front of the command or by prepending a code block with %%bash, so that is why there is a bang in fron of bash commands or %%bash in code blocks with bash commands.

So what is nmap, nmap is a flexible, extensible network scanning engine. Without even touching on the extensible bits nmap contains many sophisticated useful features. The extensibility parts makes nmap a more powerful scanning enging but extends it's capabilities in to the realm of vulnerability scanning, network toolkits and exploit delivery.

Right so first thing is RTFM which on UNIX systems is the venerable manpage.

```bash
man nmap

NMAP(1)                      Nmap Reference Guide                      NMAP(1)

NAME
       nmap - Network exploration tool and security / port scanner

SYNOPSIS
       nmap [Scan Type...] [Options] {target specification}

DESCRIPTION
       Nmap (“Network Mapper”) is an open source tool for network exploration
       and security auditing. It was designed to rapidly scan large networks,
       although it works fine against single hosts. Nmap uses raw IP packets
       in novel ways to determine what hosts are available on the network,
       what services (application name and version) those hosts are offering,
       what operating systems (and OS versions) they are running, what type of
       packet filters/firewalls are in use, and dozens of other
       ...
```

We won't include the whole think because it is really really long.

BTW about the command we just ran to dumpt the man pate into the note book, well the the -P option tells man to use formattting like cat would and just dump the page instead of paginating it for a terminsl.

What a wall of text, well with great flexibility comes great complexity.. But who reads man pages anyways, your question is probably on Stackoverflow already/... ;)

Seriously though man pages are a tremendous resource, and prehaps a value lost in today's world of Aglie development.

So ther is a lot of information about nmap out there and if your familiar with nmap and nse you might as well as stop reading now. The main objective here is to hack with something that is new and unfamilar. It is imposible to know everything about everything, however with a the right analytical skills today especially it give you the tools to travel light and get things done.

What I wanted to focus on here is the extensibility aspect, in that I am refering to NSE the nmap scripting engine...

NMap comes with a number of scrpts utilizing NSE lets take a look at one real quick...But it might help if we did this with a purpose in mind....

Lets supose we thought that developers might be using the builtin Python http-server module for development and that they are doing it in their development directory which is a git repository, they could be exposing credentials that would allow injection of malicious code in to a souce tree that could make it in to production, what a nice place to put a backdoor....

Right so how do we do this? Youre vaugly familiar with nmap, it's a port scanner that would be good for perhaps scanning for ports that have a HTTP server running. But how do we know if the server is a Python http-server? So dorking about with nmap and browsing on the interwebs you find out about this cool feature of nmap that allows scripting, and that there a lot of built in scripts in /usr/share/nmap/scripts . There is even one called http-headers that prints out HTTP headers of any HTTP server that nmap connects to
This one dumps the banner output from an number of network protocols....

Looks like a good start and it'ts easy enough to stand up a Python http-server inside a directory with a git repo to test our theory....

In [ ]:
%%bash
mkdir ~/hacking-with-nmap
cd ~/hacking-with-nmap
git init
python3 -m http.server

```
Serving HTTP on 0.0.0.0 port 8000 ...
```
 Yikes that is listening on all addresses including accessable from other machines on the network and possibly the internet.
 
 We better kill http-server and restart it, on the local loop back port.

In [ ]:
%%%bash
pkill -f "python3 -m http.server"
python3 -m http.server --bind 127.0.0.1 8000

Now we are ready to have go at the mython http-server module with the NMap NSE script http-headers

In [3]:
%%bash
nmap --script http-headers -p 80,8000,8080 127.0.0.1


Starting Nmap 7.40 ( https://nmap.org ) at 2018-05-27 22:22 EDT
Nmap scan report for localhost (127.0.0.1)
Host is up (0.00013s latency).
PORT     STATE  SERVICE
80/tcp   closed http
8000/tcp open   http-alt
| http-headers: 
|   Server: SimpleHTTP/0.6 Python/3.5.2
|   Date: Mon, 28 May 2018 02:22:13 GMT
|   Content-type: text/html; charset=utf-8
|   Content-Length: 542
|   
|_  (Request type: HEAD)
8080/tcp closed http-proxy

Nmap done: 1 IP address (1 host up) scanned in 0.28 seconds


Perfect the Python http-server populates the server field in the HTTP headers now we know how to identify potential targets.

Okay so how do we figure out how to find out if the server has a git repo inside and get the information. Uh I guess we could write a scraper but that is work...

I wonder are there any NSE scripts that do something like what we want?

In [1]:
%%%bash
ls /usr/share/nmap/scripts| grep git

http-git.nse
http-gitweb-projects-enum.nse


Hmm http-git looks interesting. Lets get some more information. From reading the man page we can use the switch --script-help to get more information on a script

In [2]:
!nmap --script-help http-git


Starting Nmap 7.40 ( https://nmap.org ) at 2018-05-28 00:32 EDT

http-git
Categories: default safe vuln
https://nmap.org/nsedoc/scripts/http-git.html
  Checks for a Git repository found in a website's document root
  /.git/<something>) and retrieves as much repo information as
  possible, including language/framework, remotes, last commit
  message, and repository description.


Perfect it does pretty much what we want, list test it out

In [3]:
!nmap --script http-git -p 8000 127.0.0.1


Starting Nmap 7.40 ( https://nmap.org ) at 2018-05-28 00:35 EDT
Nmap scan report for localhost (127.0.0.1)
Host is up (0.00014s latency).
PORT     STATE SERVICE
8000/tcp open  http-alt
| http-git: 
|   127.0.0.1:8000/.git/
|     Git repository found!
|_    Repository description: Unnamed repository; edit this file 'description' to name the...

Nmap done: 1 IP address (1 host up) scanned in 0.32 seconds


Okay so we know how to find the target servers and we know how to find the git repo if it is there. But how do we ty the two together?

Well we could parse the text output, but that seems like it would be a pain and be brittle. Hmm something from the manpage sticks out nmap can  generate XML output to a file with the -oX switch

```bash
 nmap --script http-git -p 8000 127.0.0.1 -oX -
 ```
```xml
<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE nmaprun>
<?xml-stylesheet href="file:///usr/bin/../share/nmap/nmap.xsl" type="text/xsl"?>
<!-- Nmap 7.40 scan initiated Mon May 28 00:42:10 2018 as: nmap -&#45;script http-git -p 8000 -oX - 127.0.0.1 -->
<nmaprun scanner="nmap" args="nmap -&#45;script http-git -p 8000 -oX - 127.0.0.1" start="1527482530" startstr="Mon May 28 00:42:10 2018" version="7.40" xmloutputversion="1.04">
<scaninfo type="connect" protocol="tcp" numservices="1" services="8000"/>
<verbose level="0"/>
<debugging level="0"/>
<host starttime="1527482530" endtime="1527482530"><status state="up" reason="conn-refused" reason_ttl="0"/>
<address addr="127.0.0.1" addrtype="ipv4"/>
<hostnames>
<hostname name="localhost" type="PTR"/>
</hostnames>
<ports><port protocol="tcp" portid="8000"><state state="open" reason="syn-ack" reason_ttl="0"/><service name="http-alt" method="table" conf="3"/><script id="http-git" output="&#xa;  127.0.0.1:8000/.git/&#xa;    Git repository found!&#xa;    Repository description: Unnamed repository; edit this file &apos;description&apos; to name the...&#xa;"><table key="127.0.0.1:8000/.git/">
<table key="files-found">
<elem key=".git/COMMIT_EDITMSG">false</elem>
<elem key=".git/description">true</elem>
<elem key=".git/config">true</elem>
<elem key=".git/info/exclude">true</elem>
<elem key=".gitignore">false</elem>
</table>
<elem key="repository-description">Unnamed repository; edit this file &apos;description&apos; to name the repository.&#xa;</elem>
</table>
</script></port>
</ports>
<times srtt="191" rttvar="3775" to="100000"/>
</host>
<runstats><finished time="1527482530" timestr="Mon May 28 00:42:10 2018" elapsed="0.39" summary="Nmap done at Mon May 28 00:42:10 2018; 1 IP address (1 host up) scanned in 0.39 seconds" exit="success"/><hosts up="1" down="0" total="1"/>
</runstats>
</nmaprun>
```

So that is nice, but, ug now I have to figure out how to deal with the XML (Note using the XML output is perfectly reasonable and good, just not one were going to take)

Okay so what if we could mash the two NSE scripts together and make one script that does what we want. Besides wouldn't it be cool if we could make our own NSE scripts that might be a nice think to know how to do.

The NMap NSE uses an ebeded Lua interperter for it's scrptting engine. Lua is an awesome little language similar to Javascript, small light and fast. In many ways a better Javascript than Javascript. The primary similiarities to javascript is the object systems, both Javascript and Lua use prototype based object systems.  That is to say new objects are built on top of existing objects which serve as ptototypes adding attributes and methods. There some other syntactic differences. But you  know what wether we know lua or not we only need to know enough to get this working, I'm sure the interwebs can help, enough.

Okay so lets dive right in by looking at http-headers.nse

```lua
local http = require "http"
local nmap = require "nmap"
local shortport = require "shortport"
local stdnse = require "stdnse"
local table = require "table"

description = [[
Performs a HEAD request for the root folder ("/") of a web server and displays the HTTP headers returned.
]]

---
-- @output
-- PORT   STATE SERVICE
-- 80/tcp open  http
-- | http-headers:
-- |   Date: Fri, 25 Jan 2013 17:39:08 GMT
-- |   Server: Apache/2.2.14 (Ubuntu)
-- |   Accept-Ranges: bytes
-- |   Vary: Accept-Encoding
-- |   Connection: close
-- |   Content-Type: text/html
-- |
-- |_  (Request type: HEAD)
--
--@args path The path to request, such as <code>/index.php</code>. Default <code>/</code>.
--@args useget Set to force GET requests instead of HEAD.

author = "Ron Bowes"

license = "Same as Nmap--See https://nmap.org/book/man-legal.html"

categories = {"discovery", "safe"}

portrule = shortport.http

local function fail (err) return stdnse.format_output(false, err) end

action = function(host, port)
  local path = stdnse.get_script_args(SCRIPT_NAME..".path") or "/"
  local useget = stdnse.get_script_args(SCRIPT_NAME..".useget")
  local request_type = "HEAD"
  local status = false
  local result

  -- Check if the user didn't want HEAD to be used
  if(useget == nil) then
    -- Try using HEAD first
    status, result = http.can_use_head(host, port, nil, path)
  end

  -- If head failed, try using GET
  if(status == false) then
    stdnse.debug1("HEAD request failed, falling back to GET")
    result = http.get(host, port, path)
    request_type = "GET"
  end

  if(result == nil) then
    return fail("Header request failed")
  end

  if(result.rawheader == nil) then
    return fail("Header request didn't return a proper header")
  end

  table.insert(result.rawheader, "(Request type: " .. request_type .. ")")

  return stdnse.format_output(true, result.rawheader)
end

```
Okay looks simple enough, it seems like NMap NSE scripts uses a protocol, where there are sertain specal variables that get populated and NMAP NSE uses them to do things like display documentation, preform actions durring the scan. 

Lets start by copying the code block above in a text editor saving it awith the name python-http-git.nse and change value of the description variable to describe what were going to adapt the script to do. One thing I think you've niticed is that [[ and ]] enclose or delimit a string that spans multiple lines.

```lua

description = [[
Performs a HEAD request for the root folder ("/") of a web server and displays the HTTP headers returned.
]]

```

Okay lets see if our change took

In [1]:
!nmap --script-help=./python-http-git.nse


Starting Nmap 7.40 ( https://nmap.org ) at 2018-05-28 13:41 EDT

python-http-git
Categories: discovery safe
https://nmap.org/nsedoc/scripts/python-http-git.html
  Performs a HEAD request for the root folder ("/") of a web server and display wether it is a python http-server.


Sweet that worked BTW --script-help can be used to get documentation on other nmap scripts....

Okay lets get to work, readign theought the code we can infer action is a function that gets call durring the scan if a port is open. action is given the host and port being scanned. Reading further we see that it tries to make a HTTP HEAD request. (A HTTP HEAD request returns only the headers and not the full page, browsers and spider might often make this request to decide if a page has changed since the last time the loaded it and decide wether to reload it again)

We also notice the condition where useget is checked to be nill, and reading backwards to the local declaration of useget it would appear that is an argument that can be used to make HTTP GET requests only. Cools so now we see you we might be able to pass a NSE script an argument if we wanted to.

So something interesting on line 52 is the stdnse.debug1 function looks like we can use that to print debuging output

So moving on downwe get past the conditions that checked wether to use GET requests (line 46), see if the status of the HEAD request was successful(line 52), wether the result of the request was populated (line 58), and wether it is a valid HTTP header (line 62).

So we get owards the end this is probably where we will want to make out changes...

```lua   
  table.insert(result.rawheader, "(Request type: " .. request_type .. ")")

  return stdnse.format_output(true, result.rawheader)
```
So it looks like that result.rawheader is a lua table and that it is adding a line to it. Interesting I bet that .. is used to glue pieces of strings together. Finally it is calling stdnse.format_output to format the output and return it back to nse.

Okay looking back when we ran the http-headers NSE script we get the value of the Server HTTP Heaser value returned by our puthon http-server
```
Server: SimpleHTTP/0.6 Python/3.5.2
```
Okay how do we get the value in the header? It has to be some where in the result. Okay perhaps we need to do some research. We remember a line at the top of the program

```lua
local http = require "http"
```

http must be the library that is being used, we could find it ant try to read the code (The code for the lua libraries that NSE makes avaliable is /usr/share/nmap/nselib BTW) Or we could search the interwebs for some information... Perhaps we search for 'nse http library' which should brobly find the documentation for the http library for nse (here is the link BTW https://nmap.org/nsedoc/lib/http.html )

After reading into the doc a short way it looks like the http functions return a table and one of the indexxes is head which is another table where the HEADER attributes are converted to lowercase and used as indexes.

So I think we could get at the value of the Server attribute by doing the following
```lua
result.header.server
```

Lets test this at around line 65 before the table.insert lets try to print out the value in debugging output . Add the line below to python-http-git.nse save it then runit with debug enabled
```lua
 stdnse.debug1("This is the Server attribute in the response:" .. result.header.server)
 ```

In [2]:
!nmap -d --script=./python-http-git.nse -p 8000 127.0.0.1 


Starting Nmap 7.40 ( https://nmap.org ) at 2018-05-28 14:47 EDT
--------------- Timing report ---------------
  hostgroups: min 1, max 100000
  rtt-timeouts: init 1000, min 100, max 10000
  max-scan-delay: TCP 1000, UDP 1000, SCTP 1000
  parallelism: min 0, max 0
  max-retries: 10, host-timeout: 0
  min-rate: 0, max-rate: 0
---------------------------------------------
NSE: Using Lua 5.3.
NSE: Arguments from CLI: 
NSE: Loaded 1 scripts for scanning.
NSE: Script Pre-scanning.
NSE: Starting runlevel 1 (of 1) scan.
Initiating NSE at 14:47
Completed NSE at 14:47, 0.00s elapsed
Initiating Ping Scan at 14:47
Scanning 127.0.0.1 [2 ports]
Completed Ping Scan at 14:47, 0.00s elapsed (1 total hosts)
Overall sending rates: 3809.52 packets / s.
mass_rdns: Using DNS server 192.168.1.254
mass_rdns: Using DNS server 2600:1700:840:8880::1
Initiating Connect Scan at 14:47
Scanning localhost (127.0.0.1) [1 port]
Discovered open port 8000/tcp on 127.0.0.1
Completed Connect Scan at 14:47, 0.00s elapsed (

Wow lots of text. But it is pretty cool as you look through it trying to findout if it worked you can see what nmap is doing as it preforms the scan and runs your script, pretty cool. Anyways we see that it did infact work:

    NSE: [python-http-git 127.0.0.1:8000] This is the Server attribute in the response:SimpleHTTP/0.6 Python/3.5.2

Okay so now we just need to be determine when the value of result.header.server matches "SimpleHTTP/0.6 Python/3.5.2". Of course version change so we probably want to also take this into account.

Hmm well well how do you do this in Lua? I guess we can ask the interwebs again. Perhaps we search for "lua string match"

Well the 'Lua Patterns Tutorial looks like a good place to start ( http://lua-users.org/wiki/PatternsTutorial ).
We learn that lua provides pattern matching on strings similar to regular expressions. This is primarilly preformed with the ```string match``` function, there first argument is the string and the second is the pattern.
Also we wanted to handle changing version numbers, wel ```%d``` will match any decimal and ```+``` will match the preciding value one or more times (incase the version has a decial in the double or triple digits). 

So lets fire up a lua prompt and test this out, you can do this by typing ```lua``` in your terminal.
```lua
$ lua
Lua 5.3.4  Copyright (C) 1994-2017 Lua.org, PUC-Rio
> string.match("SimpleHTTP/0.6 Python/3.5.2","Simple")
Simple
> string.match("SimpleHTTP/0.6 Python/3.5.2","SimpleHTTP/")
SimpleHTTP/
> string.match("SimpleHTTP/0.6 Python/3.5.2","SimpleHTTP/%d")
SimpleHTTP/0
> string.match("SimpleHTTP/0.6 Python/3.5.2","SimpleHTTP/%d+")
SimpleHTTP/0
> string.match("SimpleHTTP/0.6 Python/3.5.2","SimpleHTTP/%d+.%d")
SimpleHTTP/0.6
> string.match("SimpleHTTP/0.6 Python/3.5.2","SimpleHTTP/%d+.%d%s+")
SimpleHTTP/0.6 
> string.match("SimpleHTTP/0.6 Python/3.5.2","SimpleHTTP/%d+.%d%s+Python/")
SimpleHTTP/0.6 Python/
> string.match("SimpleHTTP/0.6 Python/3.5.2","SimpleHTTP/%d+.%d%s+Python/%d+.%d")
SimpleHTTP/0.6 Python/3.5
> string.match("SimpleHTTP/0.6 Python/3.5.2","SimpleHTTP/%d+.%d%s+Python/%d+.%d").%d
stdin:1: <name> expected near '%'
> string.match("SimpleHTTP/0.6 Python/3.5.2","SimpleHTTP/%d+.%d%s+Python/%d+.%d.%d")
SimpleHTTP/0.6 Python/3.5.2
> string.match("SimpleHTTP/0.6 Python/3.5.2","SimpleHTTP/%d+.%d+%s+Python/%d+.%d+.%d+")
SimpleHTTP/0.6 Python/3.5.2
> string.match("XimpleHTTP/0.6 Python/3.5.2","SimpleHTTP/%d+.%d+%s+Python/%d+.%d+.%d+")
nil
> 
```
So w notice that ```string.match`` return the matched string if it succeds and ```nil``` if it fails.
Okay so our match function call will be:
```lua
string.match(result.header.server, "SimpleHTTP/%d+.%d+%s+Python/%d+.%d+.%d+")
```

Okay lets change the bottom of the action function at like 65 to look like this

```lua

  if(result.header and result.header.server and string.match(result.header.server, "SimpleHTTP/%d+.%d+%s+Python/%d+.%d+.%d+")) then
    stdnse.debug1("The server is a python HTTP server:" .. result.header.server)
    table.insert(result.rawheader, "(Request type: " .. request_type .. ")")
    table.insert(result.rawheader, "(Server type: Python HTTP Server)")
    return stdnse.format_output(true, result.rawheader)
  else
    return fail("This is not the server we were looking for")
  end

```
Save the file and we will run the nmap script again this time without the debuging and also we will not specify ports so perhaps we will see if it is excluding other http servers that may be running (like your Jupyter note book if your following along.

In [3]:
!nmap --script=./python-http-git.nse -p 80,8000,8888 127.0.0.1 


Starting Nmap 7.40 ( https://nmap.org ) at 2018-05-28 15:25 EDT
Nmap scan report for localhost (127.0.0.1)
Host is up (0.00022s latency).
PORT     STATE  SERVICE
80/tcp   closed http
8000/tcp open   http-alt
| python-http-git: 
|   Server: SimpleHTTP/0.6 Python/3.5.2
|   Date: Mon, 28 May 2018 19:25:49 GMT
|   Content-type: text/html; charset=utf-8
|   Content-Length: 668
|   
|   (Request type: HEAD)
|_  (Server type: Python HTTP Server)
8888/tcp open   sun-answerbook

Nmap done: 1 IP address (1 host up) scanned in 0.42 seconds


Sweet, I love it when a plan comes together.....
So we can now identify python http-server servers and have accomplished half of our objective.

The other half of out objective is to be able to determine if there is a git repo exposed and if possible snarf any credentials and other useful information....

That will have to wait for another day....
